# Convert Coco JSON Annotations to YOLO TXT Files
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [13]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import sys
!{sys.executable} -m pip install pylabel
from pylabel import importer

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# YOLO Format

All the bounding boxes are placed in a `.txt`-file for each image in the dataset.

The order of attributes in each row is ```[class x_centered y_centered width height]```, where all of the values are normalized in the range of [0...1]

The centered values is the center values in both width (x) and height (y) direction, and the width and height tell the size of the bounding box.

An example of such a file:
```
0 0.0172 0.8358 0.0315 0.0347
0 0.1301 0.9474 0.0441 0.0490
0 0.1054 0.9319 0.0936 0.0891
0 0.0275 0.9392 0.0308 0.0311
```


## Import coco annotations 
First we will import annotations from the coco dataset, which are in coco json format. 

In [14]:
#Specify path to the coco.json file and images, if in another folder than annotations
TRAIN = False
# dataset_path = "../datasets/tdt4265"
dataset_path = "../../../../../work/datasets/tdt4265_2022_updated"
if TRAIN:
    path_to_annotations = os.path.join(dataset_path, "train_annotations.json")
    path_to_images = os.path.join(dataset_path, "images/train")
else:
    path_to_annotations = os.path.join(dataset_path, "val_annotations.json")
    path_to_images = os.path.join(dataset_path, "images/train")

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="labeled_data_coco_format")
dataset.df.head(1)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,1020.42,65.4,...,[],0,,,,1,car,,,1


## Analyze annotations
Pylabel can calculate basic summary statisticts about the dataset such as the number of files and the classes. 
The dataset is stored as a pandas frame so the developer can do additional exploratory analysis on the dataset. 

In [15]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")


Number of images: 301
Number of classes: 4
Classes:['car', 'bus', 'person', 'rider']
Class counts:
person    3699
car       1437
rider      315
bus         93
Name: cat_name, dtype: int64
Path to annotations:
../../../../../work/datasets/tdt4265_2022_updated


## Visualize Annotations 
You can render the bounding boxes for your image to inspect them and confirm that they imported correctly.  

In [16]:
img_id = 1 #0
selected_image_df = dataset.df.loc[dataset.df['img_id'] == img_id]
print(selected_image_df)

                                           img_folder  \
id                                                      
0   ../../../../../work/datasets/tdt4265_2022_upda...   
1   ../../../../../work/datasets/tdt4265_2022_upda...   
2   ../../../../../work/datasets/tdt4265_2022_upda...   
3   ../../../../../work/datasets/tdt4265_2022_upda...   
4   ../../../../../work/datasets/tdt4265_2022_upda...   
5   ../../../../../work/datasets/tdt4265_2022_upda...   
6   ../../../../../work/datasets/tdt4265_2022_upda...   
7   ../../../../../work/datasets/tdt4265_2022_upda...   
8   ../../../../../work/datasets/tdt4265_2022_upda...   
9   ../../../../../work/datasets/tdt4265_2022_upda...   
10  ../../../../../work/datasets/tdt4265_2022_upda...   
11  ../../../../../work/datasets/tdt4265_2022_upda...   
12  ../../../../../work/datasets/tdt4265_2022_upda...   
13  ../../../../../work/datasets/tdt4265_2022_upda...   
14  ../../../../../work/datasets/tdt4265_2022_upda...   
15  ../../../../../work/dataset

In [17]:
import cv2
import os

# Open and resize image
img = cv2.imread(os.path.join(dataset_path, selected_image_df["img_filename"][0]))
WIDTH = selected_image_df["img_width"][0]
HEIGHT = selected_image_df["img_height"][0]
print(WIDTH, HEIGHT)

# Print bounding boxes
for index, row in selected_image_df.iterrows():
    x = int(row["ann_bbox_xmin"])
    y = int(row["ann_bbox_ymin"])
    w = int(row["ann_bbox_xmax"])
    h = int(row["ann_bbox_ymax"])
    cv2.rectangle(img, (x, y), (w, h), (0, 0, 255), 2)
cv2.imwrite(f"./labeled_im{img_id}.png", img)

1024 128


True

In [18]:
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,1020.42,65.40,...,[],0,,,,1,car,,,1
1,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,983.18,58.35,...,[],0,,,,7,person,,,1
2,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,534.86,68.97,...,[],0,,,,1,car,,,1
3,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,471.78,69.62,...,[],0,,,,8,rider,,,1
4,../../../../../work/datasets/tdt4265_2022_upda...,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,469.35,69.90,...,[],0,,,,8,rider,,,1


# Export to Yolo v5
The PyLabel exporter will export all of the annotations in the dataframe to the desired target format.
Yolo creates one text file for each image in the dataset. 

In [20]:
dataset.export.ExportToYoloV5(output_path="labels/")

['dataset.yaml',
 'labels/images/val/trip007_glos_Video00003_54.txt',
 'labels/images/val/trip007_glos_Video00003_55.txt',
 'labels/images/val/trip007_glos_Video00003_56.txt',
 'labels/images/val/trip007_glos_Video00003_57.txt',
 'labels/images/val/trip007_glos_Video00003_58.txt',
 'labels/images/val/trip007_glos_Video00003_59.txt',
 'labels/images/val/trip007_glos_Video00003_60.txt',
 'labels/images/val/trip007_glos_Video00003_61.txt',
 'labels/images/val/trip007_glos_Video00003_62.txt',
 'labels/images/val/trip007_glos_Video00003_63.txt',
 'labels/images/val/trip007_glos_Video00003_64.txt',
 'labels/images/val/trip007_glos_Video00003_65.txt',
 'labels/images/val/trip007_glos_Video00003_66.txt',
 'labels/images/val/trip007_glos_Video00003_67.txt',
 'labels/images/val/trip007_glos_Video00003_68.txt',
 'labels/images/val/trip007_glos_Video00003_69.txt',
 'labels/images/val/trip007_glos_Video00003_70.txt',
 'labels/images/val/trip007_glos_Video00003_71.txt',
 'labels/images/val/trip007_g